# Register and export agar pad movies

Loads ND2 file, does registration and exports data to disk.
This was developed for 2D, multi-channel timelapse data.

In [13]:
#next line is required for Napari
%gui qt

#main data analysis packages
import numpy as np
import pandas as pd 

#image viewer
import napari

#out of memory computation
import dask.array as da
import dask

#path handling
import pathlib
import nd2

#file handling
import h5py

#image registration
from registration import register_movie

#dask cash
from dask.cache import Cache
cache = Cache(4e9)  # Leverage 4 GB of memory
cache.register()    # Turn cache on globally

## Load data

In [15]:
!open "smb://unibasel.ads.unibas.ch/BZ/RG van Vliet/ScientificData"

In [19]:
#SET path to ND2 file
expname = "20230427"
data_path = pathlib.Path("/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/2023-04-27-agar-pad/")
process_path = pathlib.Path('/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/')
data_file_path = pathlib.Path("/Users/simonvanvliet/TempData/DataFiles")

#create file names
process_dir = process_path / f"{expname}-agar-pad-processed"
process_dir.mkdir(exist_ok=True)
data_file = data_path / f"{expname}.nd2"
metadata_path = data_file_path / f"agarpad_{expname}.csv"

In [20]:
f = nd2.ND2File(data_file)
raw_data = f.to_dask()
#see metadata https://pypi.org/project/nd2/
print(f.metadata.channels)
print(f.experiment)
print(f.sizes)
f.close()

[Channel(channel=ChannelMeta(name='mCherry_GR', index=0, colorRGB=11519, emissionLambdaNm=629.5, excitationLambdaNm=575.0), loops=LoopIndices(NETimeLoop=None, TimeLoop=0, XYPosLoop=1, ZStackLoop=None), microscope=Microscope(objectiveMagnification=60.0, objectiveName='Plan Apo λ 60x Oil Ph3 DM', objectiveNumericalAperture=1.4, zoomMagnification=1.0, immersionRefractiveIndex=1.515, projectiveMagnification=None, pinholeDiameterUm=None, modalityFlags=['fluorescence']), volume=Volume(axesCalibrated=[True, True, False], axesCalibration=[0.108333333333333, 0.108333333333333, 1.0], axesInterpretation=(<AxisInterpretation.distance: 'distance'>, <AxisInterpretation.distance: 'distance'>, <AxisInterpretation.distance: 'distance'>), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=[-0.9991757912166891, 0.04059234221504424, -0.04059234221504424, -0.9991757912166891], componentCount=1, componentDataType='unsigned', voxelCount=[2048, 2044, 1], componentMaxima=[0

## Visualize with Napari

In [7]:
viewer = napari.view_image(raw_data,
            channel_axis=2,
            name=["SA1-RFP", "SA2-GFP", "PA-Phase"],
            colormap=["red", "green", "gray"])
napari.run()

## Select maximum frame for each position and create metadata dataframe 

use the 0B_create_metadata.ipynb to create and save the metadata

In [10]:
#load metadata
df = pd.read_csv(metadata_path, index_col=0)

#extract max frames
max_frames = np.full(raw_data.shape[1],np.nan)
for pos in range(raw_data.shape[1]):
    if f"pos{pos:03d}" in df.index:
        max_frames[pos] = df.loc[f"pos{pos:03d}","max_frame"].astype(int)

## Register and Export Movie

Warning: this can take a while to run

In [11]:
reg_data,translation = register_movie(raw_data, process_dir, expname, save_images=True, max_frames=max_frames)

## Visualize registration with Napari

In [12]:
viewer.add_image(reg_data, 
                 channel_axis=2,
                name=["SA1-RFP", "SA2-GFP", "PA-Phase"],
                colormap=["red", "green", "gray"])
napari.run()
napari.run()